# Pre-Processing
1) Clean data by removing rows with > 50% missing info <br>
2) Find the most informative features <br>
3) Split the data into test and train


In [54]:
library(caret)
library(corrplot)
library(biomaRt)
library(GeneAnswers) # used to get gene names from entrez IDs

## Load data
The first column is expected to be sample ID <br>
The second column is expected to be response

In [2]:
setwd("/home/jp/ICP_Responders/FinalTables")
expression <- read.csv("Final_table_response_and_expression.csv", na.strings = '..', stringsAsFactors = F)
names(expression) <- sub("^X", "", names(expression))
expression[ expression == "NA" ] <- NA
expression[,2:ncol(expression)] <- lapply(expression[, 2:ncol(expression)], as.numeric)
expression[50:60, 1:10]

,Patient,Response,3920,345611,3929,54210,3716,10454,3557,3556
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
50,31,1,NA,NA,NA,NA,NA,NA,NA,NA
51,32,1,NA,NA,NA,NA,NA,NA,NA,NA
52,33,1,NA,NA,NA,NA,NA,NA,NA,NA
53,34,1,-0.24227274,0.074304976,-0.3511685,0.54428208,-0.15006954,-0.07183968,0.20022650,-1.182660036
54,35,1,-0.48479782,0.159405465,0.4199815,-2.22883720,0.24650857,-0.38904130,1.82423349,-1.343065827
55,36,1,0.74085490,0.003467014,0.3333496,-0.03988917,-0.21346769,0.54476821,-1.69969237,0.483498059
56,37,1,NA,NA,NA,NA,NA,NA,NA,NA
57,38,1,0.81259519,-0.048404344,-0.2250873,-0.29189077,0.44538081,0.21789661,-0.08321449,1.567718745
58,39,0,NA,NA,NA,NA,NA,NA,NA,NA


## Clean data

In [3]:
# Check which column has > 50% NA values
countNA <- function(x=NULL,cutOff=NULL){
  output<-FALSE
  perc<-sum(is.na(x))*100/length(x)
  if(perc>cutOff){output<-TRUE}
  output  
}
col_nas <- apply(expression,2,function(x){countNA(x, 50)})
cat("Columns with NAs > 50% = ", sum(col_nas), "\n")
# all columns have <50% NAs

# Check which rows has > 50% NA values
row_nas <- apply(expression,1,function(x){countNA(x, 50)})
cat("Rows with NAs > 50% = ", sum(row_nas), "\n")

# 43 rows have have >50% NAs, removing them
expr_filtered <- expression[-which(row_nas %in% TRUE),]

cat("Dimensions of the filtered dataset = ", dim(expr_filtered))

Columns with NAs > 50% =  0 
Rows with NAs > 50% =  43 
Dimensions of the filteres dataset =  161 676

## Look for near zero variance and remove those columns

In [4]:
nzv <- nearZeroVar(expr_filtered[3:ncol(expr_filtered)], saveMetrics= TRUE)
nzv[which(nzv$zeroVar %in% TRUE), ]

# All features were retained and there was no filtering due to near zero variance

freqRatio,percentUnique,zeroVar,nzv
<dbl>,<dbl>,<lgl>,<lgl>


## Look for correlation and remove highly correlated columns

In [5]:
# find attributes that are highly corrected (ideally >0.75)
tmp <- expr_filtered
tmp[is.na(tmp)] <- 0
expr_corr <-  cor(tmp[,3:ncol(tmp)]) 
highlyCorrelated <- findCorrelation(expr_corr, cutoff=0.75, names=TRUE, verbose=TRUE)
expr_rmcorr <- expr_filtered[, -which(colnames(expr_filtered) %in% highlyCorrelated)]


 Combination row 18 and column 19 is above the cut-off, value = 0.797 
 	 Flagging column 18 
 Combination row 18 and column 20 is above the cut-off, value = 0.755 
 	 Flagging column 18 
 Combination row 20 and column 22 is above the cut-off, value = 0.832 
 	 Flagging column 22 
 Combination row 20 and column 23 is above the cut-off, value = 0.801 
 	 Flagging column 23 
 Combination row 22 and column 23 is above the cut-off, value = 0.954 
 	 Flagging column 22 
 Combination row 18 and column 24 is above the cut-off, value = 0.768 
 	 Flagging column 18 
 Combination row 19 and column 24 is above the cut-off, value = 0.838 
 	 Flagging column 24 
 Combination row 19 and column 25 is above the cut-off, value = 0.759 
 	 Flagging column 25 
 Combination row 24 and column 25 is above the cut-off, value = 0.929 
 	 Flagging column 25 
 Combination row 16 and column 26 is above the cut-off, value = 0.856 
 	 Flagging column 16 
 Combination row 16 and column 31 is above the cut-off, val

## Linear dependencies
NOTE: Linear Dependencies were not used because
1) A large number of features from columns 157 to 400 were flagged as linearly dependent to columns 1 to 156 <br>
2) We will rely on feature importance and the robustness of the model selected to identify the most important features hence linear dependencies can become redundant

In [87]:
# tmp <- expr_rmcorr[, 3:ncol(expr_rmcorr)]
# tmp[is.na(tmp)] <- 0
# comboInfo <- findLinearCombos(tmp) 
# comboInfo
# rmLnCmb <- colnames(tmp[,comboInfo$remove])
# expr_rmLnCmb <- expr_rmcorr[,-which(colnames(expr_rmcorr) %in% rmLnCmb)]
cat("Started with dimension = ", dim(expression), "\n")
cat("Post 50% NA filtering in rows and columns the dimension is", dim(expr_filtered), "\n")
cat("Post filtering highly correlated columns the dimension is", dim(expr_rmcorr), "\n")
# cat("Post removing linearly dependent columns the dimension is", dim(expr_rmLnCmb))

Started with dimension =  204 676 
Post 50% NA filtering in rows and columns the dimension is 161 676 
Post filtering highly correlated columns the dimension is 161 495 


# Feature Selection


`How does var imp work, significance?`<br>
For classification, ROC curve analysis is conducted on each predictor. For two class problems, a series of cutoffs is applied to the predictor data to predict the class. The sensitivity and specificity are computed for each cutoff and the ROC curve is computed. The trapezoidal rule is used to compute the area under the ROC curve. This area is used as the measure of variable importance.

In [89]:
# define a resampling approach for caret where data is divided into 8 random subsets and prediction is done on 
# 1 using the remaining 7. This approach is repeated thrice
control <- trainControl(method="repeatedcv", number=8, repeats=3)
mod_inp_mat <- expr_rmcorr[, 2:ncol(expr_rmcorr)]
mod_inp_mat$Response <- as.factor(mod_inp_mat$Response)

## glmNet
A generalized linear model (GLM) is a flexible generalization of ordinary linear regression that allows for the response variable to have an error distribution other than the normal distribution. The GLM generalizes linear regression by allowing the linear model to be related to the response variable via a link function and by allowing the magnitude of the variance of each measurement to be a function of its predicted value.

Advantages Disadvantages?

In [18]:
m.glm <- train(Response~., data=mod_inp_mat, 
                  method="glmnet", 
                  trControl=control,
                  preProcess = c("scale", "center"),
                  na.action = na.omit
                 )


In [19]:
# estimate variable importance
glm.imp <- varImp(m.glm, scale=TRUE)$importance
rownames(glm.imp) <- gsub("`", "", rownames(glm.imp))
glm.imp$Name <- rownames(glm.imp)
# summarize importance
glm.imp <- glm.imp[order(glm.imp$Overall, decreasing=TRUE),]
colnames(glm.imp) <- c("Score", "Name")
head(glm.imp)

,Score,Name
,<dbl>,<chr>
2919,100.00000,2919
9636,77.59147,9636
54674,61.54440,54674
7128,60.85943,7128
6374,58.17678,6374
56477,51.98825,56477


## SVM
Support vector machines (SVMs) are a set of supervised learning methods used for classification, regression and outliers detection.

Advantages:<br>
1) Effective in high dimensional spaces. <br>
2) Still effective in cases where number of dimensions is greater than the number of samples.

Disadvantages:<br> 
??????

In [20]:
m.svm <- train(Response~., data=mod_inp_mat, 
                  method="svmLinear2", 
                  trControl=control,
                  preProcess = c("scale", "center"),
                  na.action = na.omit
                 )

In [21]:
# estimate variable importance
svm.imp <- varImp(m.svm, scale=TRUE)$importance
svm.imp$Name <- rownames(svm.imp)
# summarize importance
svm.imp <- svm.imp[order(svm.imp$X0, decreasing=TRUE),]
svm.imp$X0 <- NULL
colnames(svm.imp) <- c("Score", "Name")
head(svm.imp)

,Score,Name
,<dbl>,<chr>
6504,100.00000,6504
3458,87.24062,3458
29851,82.38411,29851
7128,78.05740,7128
4210,72.75938,4210
1493,71.39073,1493


## Neural Network

In [22]:
m.nnet <- train(Response~., data=mod_inp_mat, 
                  method="nnet", 
                  trControl=control,
                  preProcess = c("scale", "center"),
                  na.action = na.omit
                 )

# weights:  159
initial  value 93.831261 
iter  10 value 60.012316
iter  20 value 49.896523
iter  30 value 46.701671
iter  40 value 46.377476
iter  50 value 46.356821
iter  60 value 44.863671
iter  70 value 44.722732
iter  80 value 44.720711
iter  90 value 44.719833
iter 100 value 44.718972
final  value 44.718972 
stopped after 100 iterations
# weights:  475
initial  value 98.648014 
iter  10 value 46.103538
iter  20 value 27.846053
iter  30 value 20.782395
iter  40 value 16.699598
iter  50 value 16.301983
iter  60 value 16.007028
iter  70 value 15.756625
iter  80 value 15.708689
iter  90 value 15.563915
iter 100 value 15.544106
final  value 15.544106 
stopped after 100 iterations
# weights:  791
initial  value 96.313537 
iter  10 value 26.953059
iter  20 value 10.540479
iter  30 value 9.229052
iter  40 value 8.773808
iter  50 value 8.655397
iter  60 value 8.502945
iter  70 value 8.498573
iter  80 value 8.471514
iter  90 value 8.470963
iter 100 value 8.469883
final  value 8.469883 
sto

initial  value 93.625009 
iter  10 value 70.157703
iter  20 value 55.094027
iter  30 value 52.211724
iter  40 value 45.640324
iter  50 value 44.591335
iter  60 value 42.875284
iter  70 value 41.999657
iter  80 value 41.994116
iter  90 value 41.988360
iter 100 value 41.983253
final  value 41.983253 
stopped after 100 iterations
# weights:  475
initial  value 104.009264 
iter  10 value 56.684815
iter  20 value 34.762767
iter  30 value 29.911916
iter  40 value 27.197248
iter  50 value 23.831214
iter  60 value 23.704848
iter  70 value 23.674896
iter  80 value 22.506316
iter  90 value 22.030709
iter 100 value 21.928432
final  value 21.928432 
stopped after 100 iterations
# weights:  791
initial  value 97.731571 
iter  10 value 49.777906
iter  20 value 27.027775
iter  30 value 19.281132
iter  40 value 14.775559
iter  50 value 12.360650
iter  60 value 12.148962
iter  70 value 11.934052
iter  80 value 11.918964
iter  90 value 11.677151
iter 100 value 11.336193
final  value 11.336193 
stopped a

initial  value 153.009640 
iter  10 value 51.840493
iter  20 value 33.491929
iter  30 value 23.147180
iter  40 value 18.875394
iter  50 value 16.278332
iter  60 value 14.970842
iter  70 value 13.891981
iter  80 value 13.698925
iter  90 value 13.663845
iter 100 value 13.656433
final  value 13.656433 
stopped after 100 iterations
# weights:  791
initial  value 109.825798 
iter  10 value 51.865560
iter  20 value 26.347001
iter  30 value 15.478683
iter  40 value 13.397531
iter  50 value 12.699415
iter  60 value 12.501391
iter  70 value 12.335969
iter  80 value 12.295216
iter  90 value 12.230646
iter 100 value 12.185972
final  value 12.185972 
stopped after 100 iterations
# weights:  159
initial  value 123.083634 
iter  10 value 73.174247
iter  20 value 48.729086
iter  30 value 45.664068
iter  40 value 42.074654
iter  50 value 42.005704
iter  60 value 42.000207
iter  70 value 41.996327
iter  80 value 41.992695
iter  90 value 41.988144
iter 100 value 41.962798
final  value 41.962798 
stopped

initial  value 136.179324 
iter  10 value 57.974301
iter  20 value 40.724206
iter  30 value 37.752036
iter  40 value 35.590276
iter  50 value 30.799159
iter  60 value 29.855740
iter  70 value 29.577253
iter  80 value 29.367376
iter  90 value 28.935414
iter 100 value 28.759736
final  value 28.759736 
stopped after 100 iterations
# weights:  791
initial  value 127.997169 
iter  10 value 46.368030
iter  20 value 20.405121
iter  30 value 14.085509
iter  40 value 10.670548
iter  50 value 10.276592
iter  60 value 10.248971
iter  70 value 9.229313
iter  80 value 7.057965
iter  90 value 6.772371
iter 100 value 6.556405
final  value 6.556405 
stopped after 100 iterations
# weights:  159
initial  value 101.355111 
iter  10 value 57.135854
iter  20 value 42.141998
iter  30 value 34.066770
iter  40 value 31.456733
iter  50 value 26.397247
iter  60 value 25.731808
iter  70 value 22.245937
iter  80 value 22.196071
iter  90 value 22.194689
final  value 22.194668 
converged
# weights:  475
initial  va

initial  value 94.172331 
iter  10 value 57.082378
iter  20 value 23.631308
iter  30 value 13.672968
iter  40 value 10.016643
iter  50 value 8.483655
iter  60 value 8.350363
iter  70 value 7.904519
iter  80 value 7.852482
iter  90 value 3.420768
iter 100 value 2.422304
final  value 2.422304 
stopped after 100 iterations
# weights:  159
initial  value 102.945155 
iter  10 value 64.468889
iter  20 value 56.755617
iter  30 value 54.451792
iter  40 value 49.485472
iter  50 value 49.392890
iter  60 value 49.372985
iter  70 value 49.367878
iter  80 value 47.080212
iter  90 value 47.079022
iter 100 value 47.078190
final  value 47.078190 
stopped after 100 iterations
# weights:  475
initial  value 116.869288 
iter  10 value 65.700000
iter  20 value 44.779718
iter  30 value 43.054033
iter  40 value 42.610440
iter  50 value 42.096840
iter  60 value 41.950133
iter  70 value 41.734946
iter  80 value 41.581461
iter  90 value 41.464125
iter 100 value 41.450382
final  value 41.450382 
stopped after 1

initial  value 104.824255 
iter  10 value 74.509966
iter  20 value 61.793537
iter  30 value 59.474039
iter  40 value 56.877531
iter  50 value 53.320263
iter  60 value 49.799744
iter  70 value 47.791009
iter  80 value 46.248454
iter  90 value 46.195343
iter 100 value 44.618146
final  value 44.618146 
stopped after 100 iterations
# weights:  475
initial  value 94.955500 
iter  10 value 58.497871
iter  20 value 22.842074
iter  30 value 20.546789
iter  40 value 20.434035
iter  50 value 20.080646
iter  60 value 20.048684
iter  70 value 19.442450
iter  80 value 19.271541
iter  90 value 19.090526
iter 100 value 18.911479
final  value 18.911479 
stopped after 100 iterations
# weights:  791
initial  value 121.082973 
iter  10 value 49.824866
iter  20 value 22.017961
iter  30 value 18.730591
iter  40 value 16.730500
iter  50 value 14.280842
iter  60 value 12.532199
iter  70 value 12.217336
iter  80 value 12.044792
iter  90 value 11.564672
iter 100 value 11.441872
final  value 11.441872 
stopped 

initial  value 93.412747 
iter  10 value 59.754651
iter  20 value 41.580309
iter  30 value 36.939075
iter  40 value 31.878359
iter  50 value 27.149471
iter  60 value 22.101710
iter  70 value 20.238224
iter  80 value 19.157579
iter  90 value 19.015152
iter 100 value 19.013232
final  value 19.013232 
stopped after 100 iterations
# weights:  475
initial  value 131.492956 
iter  10 value 59.518891
iter  20 value 31.192176
iter  30 value 21.431304
iter  40 value 16.583790
iter  50 value 14.564945
iter  60 value 13.939106
iter  70 value 13.500667
iter  80 value 13.356834
iter  90 value 13.237764
iter 100 value 13.229167
final  value 13.229167 
stopped after 100 iterations
# weights:  791
initial  value 116.211952 
iter  10 value 59.979237
iter  20 value 32.400888
iter  30 value 19.805606
iter  40 value 14.251722
iter  50 value 12.949929
iter  60 value 12.500832
iter  70 value 12.286954
iter  80 value 12.155226
iter  90 value 12.086149
iter 100 value 11.995344
final  value 11.995344 
stopped 

initial  value 104.580198 
iter  10 value 71.147881
iter  20 value 61.098090
iter  30 value 44.306258
iter  40 value 41.318058
iter  50 value 41.253695
iter  60 value 38.259133
iter  70 value 38.242052
iter  80 value 38.237597
iter  90 value 38.234465
iter 100 value 38.233338
final  value 38.233338 
stopped after 100 iterations
# weights:  475
initial  value 92.565695 
iter  10 value 38.183424
iter  20 value 21.755394
iter  30 value 19.532440
iter  40 value 18.972739
iter  50 value 18.786643
iter  60 value 17.738696
iter  70 value 14.514443
iter  80 value 13.899885
iter  90 value 13.899174
iter 100 value 13.823738
final  value 13.823738 
stopped after 100 iterations
# weights:  791
initial  value 111.594354 
iter  10 value 42.666613
iter  20 value 19.672001
iter  30 value 14.378429
iter  40 value 13.352089
iter  50 value 13.020571
iter  60 value 12.896041
iter  70 value 12.784313
iter  80 value 12.431799
iter  90 value 12.398946
iter 100 value 12.322125
final  value 12.322125 
stopped 

initial  value 96.220632 
iter  10 value 43.698621
iter  20 value 24.573919
iter  30 value 18.195341
iter  40 value 17.623180
iter  50 value 17.271041
iter  60 value 17.060348
iter  70 value 16.805399
iter  80 value 16.725820
iter  90 value 16.590187
iter 100 value 16.472807
final  value 16.472807 
stopped after 100 iterations
# weights:  791
initial  value 99.948056 
iter  10 value 46.579050
iter  20 value 17.818664
iter  30 value 10.687413
iter  40 value 10.285400
iter  50 value 10.153928
iter  60 value 10.021128
iter  70 value 9.883766
iter  80 value 9.845464
iter  90 value 8.752907
iter 100 value 7.255908
final  value 7.255908 
stopped after 100 iterations
# weights:  159
initial  value 104.005158 
iter  10 value 76.011324
iter  20 value 57.741839
iter  30 value 54.526952
iter  40 value 54.487451
iter  50 value 54.477691
iter  60 value 54.476537
iter  70 value 54.476475
final  value 54.476470 
converged
# weights:  475
initial  value 94.782498 
iter  10 value 58.601048
iter  20 val

In [23]:
# estimate variable importance
nnet.imp <- varImp(m.nnet, scale=TRUE)$importance
rownames(nnet.imp) <- gsub("`", "", rownames(nnet.imp))
nnet.imp$Name <- rownames(nnet.imp)
# summarize importance
nnet.imp <- nnet.imp[order(nnet.imp$Overall, decreasing=TRUE),]
colnames(nnet.imp) <- c("Score", "Name")
head(nnet.imp)

,Score,Name
,<dbl>,<chr>
2919,100.00000,2919
1191,88.39249,1191
6374,86.01283,6374
56477,79.20295,56477
4772,77.44874,4772
4773,74.34398,4773


## Gradient Boosting Machines (GBM)

In [33]:
library(gbm)
m.gbm <- train(Response~., data=mod_inp_mat, 
                  method="gbm", 
                  trControl=control,
                  preProcess = c("scale", "center"),
                  na.action = na.omit
                 )

Loaded gbm 2.1.8



Iter   TrainDeviance   ValidDeviance   StepSize   Improve
     1        1.3165            -nan     0.1000    0.0014
     2        1.2901            -nan     0.1000    0.0078
     3        1.2803            -nan     0.1000   -0.0101
     4        1.2646            -nan     0.1000    0.0038
     5        1.2447            -nan     0.1000    0.0033
     6        1.2298            -nan     0.1000   -0.0017
     7        1.2157            -nan     0.1000   -0.0026
     8        1.2014            -nan     0.1000   -0.0049
     9        1.1900            -nan     0.1000   -0.0013
    10        1.1812            -nan     0.1000   -0.0049
    20        1.0867            -nan     0.1000   -0.0060
    40        0.9180            -nan     0.1000    0.0025
    60        0.7966            -nan     0.1000   -0.0064
    80        0.6858            -nan     0.1000   -0.0027
   100        0.5990            -nan     0.1000   -0.0025
   120        0.5284            -nan     0.1000   -0.0026
   140        

In [34]:
# estimate variable importance
gbm.imp <- varImp(m.gbm, scale=TRUE)$importance
rownames(gbm.imp) <- gsub("`", "", rownames(gbm.imp))
gbm.imp$Name <- rownames(gbm.imp)
# summarize importance
gbm.imp <- gbm.imp[order(gbm.imp$Overall, decreasing=TRUE),]
colnames(gbm.imp) <- c("Score", "Name")
head(gbm.imp)

,Score,Name
,<dbl>,<chr>
3458,100.00000,3458
3589,78.81366,3589
6504,77.13285,6504
947,69.87038,947
8795,69.07898,8795
332,67.49739,332


## Partial Least Squares

In [35]:
m.pls <- train(Response~., data=mod_inp_mat, 
                  method="pls", 
                  trControl=control,
                  preProcess = c("scale", "center"),
                  na.action = na.omit
                 )

In [36]:
# estimate variable importance
pls.imp <- varImp(m.pls, scale=TRUE)$importance
rownames(pls.imp) <- gsub("`", "", rownames(pls.imp))
pls.imp$Name <- rownames(pls.imp)
# summarize importance
pls.imp <- pls.imp[order(pls.imp$Overall, decreasing=TRUE),]
colnames(pls.imp) <- c("Score", "Name")
head(pls.imp)


Attaching package: ‘pls’


The following object is masked from ‘package:corrplot’:

    corrplot


The following object is masked from ‘package:caret’:

    R2


The following object is masked from ‘package:stats’:

    loadings




,Score,Name
,<dbl>,<chr>
947,100.00000,947
6504,98.47985,6504
4210,96.04660,4210
2919,95.56852,2919
54674,94.74722,54674
7177,89.51057,7177


##  Cforest

In [37]:
m.cforest <- train(Response~., data=mod_inp_mat, 
                  method="cforest", 
                  trControl=control,
                  preProcess = c("scale", "center"),
                  na.action = na.omit
                 )

In [38]:
# estimate variable importance
cforest.imp <- varImp(m.cforest, scale=TRUE)$importance
rownames(cforest.imp) <- gsub("`", "", rownames(cforest.imp))
cforest.imp$Name <- rownames(cforest.imp)
# summarize importance
cforest.imp <- cforest.imp[order(cforest.imp$Overall, decreasing=TRUE),]
colnames(cforest.imp) <- c("Score", "Name")
head(cforest.imp)

,Score,Name
,<dbl>,<chr>
3458,100.00000,3458
6504,92.65308,6504
2919,81.93662,2919
1493,69.90778,1493
8794,68.99120,8794
6373,65.26205,6373


## Linear Discriminant Analysis with Stepwise Feature Selection

In [9]:
m.stepLDA <- train(Response~., data=mod_inp_mat, 
                  method="stepLDA", 
                  trControl=control,
                  preProcess = c("scale", "center"),
                  na.action = na.omit
                 )

 `stepwise classification', using 10-fold cross-validated correctness rate of method lda'.

141 observations of 156 variables in 2 classes; direction: both

stop criterion: improvement less than 5%.



correctness rate: 0.65952;  in: "`2919`";  variables (1): `2919` 

 hr.elapsed min.elapsed sec.elapsed 
      0.000       0.000       3.464 



 `stepwise classification', using 10-fold cross-validated correctness rate of method lda'.

141 observations of 156 variables in 2 classes; direction: both

stop criterion: improvement less than 5%.



correctness rate: 0.68095;  in: "`1493`";  variables (1): `1493` 

 hr.elapsed min.elapsed sec.elapsed 
      0.000       0.000       3.464 



 `stepwise classification', using 10-fold cross-validated correctness rate of method lda'.

141 observations of 156 variables in 2 classes; direction: both

stop criterion: improvement less than 5%.



...

correctness rate: 0.66667;  in: "`7128`";  variables (1): `7128` 

 hr.elapsed min.elapsed sec.elapsed 
      0.000       0.000       3.413 



 `stepwise classification', using 10-fold cross-validated correctness rate of method lda'.

141 observations of 156 variables in 2 classes; direction: both

stop criterion: improvement less than 5%.



correctness rate: 0.65286;  in: "`29851`";  variables (1): `29851` 

 hr.elapsed min.elapsed sec.elapsed 
      0.000       0.000       3.409 



 `stepwise classification', using 10-fold cross-validated correctness rate of method lda'.

161 observations of 156 variables in 2 classes; direction: both

stop criterion: improvement less than 5%.



correctness rate: 0.65846;  in: "`7128`";  variables (1): `7128` 

 hr.elapsed min.elapsed sec.elapsed 
      0.000       0.000       3.474 



In [12]:
# estimate variable importance
stepLDA.imp <- varImp(m.stepLDA, scale=TRUE)$importance
rownames(stepLDA.imp) <- gsub("`", "", rownames(stepLDA.imp))
stepLDA.imp$Name <- rownames(stepLDA.imp)
# summarize importance
stepLDA.imp <- stepLDA.imp[order(stepLDA.imp$X0, decreasing=TRUE),]
stepLDA.imp$X0 <- NULL
colnames(stepLDA.imp) <- c("Score", "Name")
head(stepLDA.imp)

,Score,Name
,<dbl>,<chr>
6504,100.00000,6504
3458,87.24062,3458
29851,82.38411,29851
7128,78.05740,7128
4210,72.75938,4210
1493,71.39073,1493


## Naive Bayes

In [15]:
m.nb <- train(Response~., data=mod_inp_mat, 
                  method="naive_bayes", 
                  trControl=control,
                  preProcess = c("scale", "center"),
                  na.action = na.omit
                 )

In [16]:
# estimate variable importance
nb.imp <- varImp(m.nb, scale=TRUE)$importance
rownames(nb.imp) <- gsub("`", "", rownames(nb.imp))
nb.imp$Name <- rownames(nb.imp)
# summarize importance
nb.imp <- nb.imp[order(nb.imp$X0, decreasing=TRUE),]
nb.imp$X0 <- NULL
colnames(nb.imp) <- c("Score", "Name")
head(nb.imp)

,Score,Name
,<dbl>,<chr>
6504,100.00000,6504
3458,87.24062,3458
29851,82.38411,29851
7128,78.05740,7128
4210,72.75938,4210
1493,71.39073,1493


# Rank Analysis

In [48]:
# getRank adds a rank column to the input matrix according to varImp scores
getRank <- function(tmp=NULL, cname=NULL){
    tmp[,cname] <- rank(-tmp$Score)
    tmp$Score <- NULL
    tmp
}

glm.rank <- getRank(glm.imp, "glm.Rank")
svm.rank <- getRank(svm.imp, "svm.Rank")
nnet.rank <- getRank(nnet.imp, "nnet.Rank")
gbm.rank <- getRank(gbm.imp, "gbm.Rank")
pls.rank <- getRank(pls.imp, "pls.Rank")
cforest.rank <- getRank(cforest.imp, "cforest.Rank")
stepLDA.rank <- getRank(stepLDA.imp, "stepLDA.Rank")
nb.rank <- getRank(nb.imp, "nb.Rank")

all.rank <- Reduce(function(x,y) merge(x,y, by="Name"),list(glm.rank, svm.rank, nnet.rank, gbm.rank,
                                                             pls.rank, cforest.rank, stepLDA.rank, nb.rank))
                   
colnames(all.rank)[1] <- "Gene_ID" 
all.rank$Gene_Name <- getSymbols(all.rank$Gene_ID, 'org.Hs.eg.db')
# all.rank is the merged matrix containing gene IDs as the first column, gene names as the last column and 
# ranks for all the models as intermediate columns

# to find all genes over the threshold
rank.threshold <- 50
rank.clmn <- grep("Rank", colnames(all.rank))
all.rank$Threshold.50 <- apply(all.rank[, rank.clmn], 1, function(x){sum(x<rank.threshold)})
all.rank <- all.rank[order(all.rank$Threshold.50, decreasing=TRUE),]

In [64]:
# check where a gene was removed
gene_names <- c("958") # CD40
which(colnames(expression) %in% gene_names)
which(colnames(expr_filtered) %in% gene_names)
which(colnames(expr_rmcorr) %in% gene_names)
which(colnames(expr_rmLnCmb) %in% gene_names)

[1] 642

[1] 642

[1] 471

integer(0)

In [71]:
# to do a wikipathway search
temp_rank <- all.rank[all.rank$Threshold.50 >= 4, "Gene_Name"]
for (x in seq_along(temp_rank)){
    cat(temp_rank[x], "\n")
}

CXCL1 
IFNG 
LRRN3 
SLAMF1 
TAP2 
TNFRSF10C 
CD34 
MEFV 
TNFRSF11B 
CXCL11 
TNFAIP3 
TRAF3 
CTLA4 
ICOS 
ICAM1 
CCL28 
BMI1 
COG7 
FYN 
HLA-DQB1 
IRGM 
IL2RA 
IL11 
IRF4 
ITGA2 
BST2 
TAPBP 
TNF 
C2 
FEZ1 
ADA 
EBI3 
SPINK5 
NLRP3 
ABL1 
XCR1 
MR1 
HSD11B1 
KLRF1 
BCL2 
RORC 
CXCL5 
CARD9 
SSX1 
TPSAB1 
CD28 
ISG15 


In [73]:
length(temp_rank)

[1] 47

`TODO:` <br>
Find genes related to antipd1, link to the papers, make list of genes as a vector before pre processing<br>
Borda method to vote and rank features <br>
Find common pathways<br>
Add descriptions and advantages/disadvantages for all ml models

set seed<br>

In [ ]:
#merge rank using Borda method
library(votesys)
tmp<-as.matrix(t(data[,grep("rank",colnames(data))]))
colnames(tmp)<-data$gene
vote <- create_vote(tmp,xtype=1)
res.borda <- borda_method(vote)
merged.rank<-data.frame(gene=names(res.borda$other_info$count_min),rank.borda=res.borda$other_info$count_min,stringsAsFactors=F)

In [90]:
save.image()

In [76]:
# #parameter tuning
# modelLookup('lvq')
# # design the parameter tuning grid
# grid <- expand.grid(size=c(5,10,20,50), k=c(1,2,3,4,5))
# # train the model
# model <- train(Species~., data=iris, method="lvq", trControl=control, tuneGrid=grid)

In [77]:
# saveRds(tmp, "tmp.rds")
# # load by giving path
# # tmp <- readRds("path")
# # R
